<a href="https://colab.research.google.com/github/Gongsoo/Python3/blob/main/mask_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

!git clone https://github.com/Pseudo-Lab/Tutorial-Book-Utils

Cloning into 'Tutorial-Book-Utils'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 36 (delta 12), reused 17 (delta 5), pack-reused 0
Unpacking objects: 100% (36/36), 9.65 KiB | 1.21 MiB/s, done.


In [5]:
!python Tutorial-Book-Utils/PL_data_loader.py --data FaceMaskDetection


Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1pJtohTc9NGNRzHj5IsySR39JIRPfkgD3 

Face Mask Detection.zip download complete!


In [8]:

!unzip -q Face\ Mask\ Detection.zip

unzip:  cannot find or open Face Mask Detection.zip, Face Mask Detection.zip.zip or Face Mask Detection.zip.ZIP.
